In [4]:
import pandas as pd

data = pd.read_csv("D:\Sem 5\Data Analytics\Assignments\Project\Accidents_categorical.csv")



In [5]:
def preprop(data, to_drop=[], na_thresh = 0.25):
    log = []
    for i in to_drop:                           # Explicitly drop specified columns (intuition and logical reasoning) 
        data = data.drop(i, axis=1)
        log.append("Dropped column " + i + " as requested")

    report = getStats(data)
    # print(report)
    replace_index = {}
    for i in report:
        # print(i)
        if i == "uncertain":            # report has a class of uncertain columns; Do nothing to them
            continue
        
        # If numerical column and less than threshold percentage is empty, fill it with mean of column
        if report[i]["type"] == 'num' and report[i]["na"] > 0 and report[i]["na"] < na_thresh :
            data[i] = data[i].fillna(data[i].mean())
            log.append(i + " : replaced Nan/Na with mean")
            
        #If in any column, more than threshold percentage is empty, drop such columns
        elif (report[i]["type"] == 'num' or report[i]["type"] == 'cat') and report[i]["na"] >= na_thresh :
            # print(data.columns)
            data = data.drop(i, axis=1)
            log.append("Dropped column " + i + " as Nan/Na ratio (" + str(report[i]["na"]) + ") > " + str(na_thresh))
            
            
        # If column is categorical, encode them with numerical classes    
        if report[i]["type"] == 'cat' and i in data.columns:
            replace_index[i] = getEncodings(data[i])
            log.append("Encoded column " + i)

    data = data.replace(replace_index)


    return data, log, replace_index # Returning cleaned data, log of changes made and categorical encodings dict (if any)



def getEncodings(o):
    o = list(set(o))
    o = sorted(o)
    enc = {cls: ind for ind, cls in enumerate(o)}
    return enc


def getStats(data):
    cols = data.columns
    report = {}
    report['uncertain'] = []
    for i in cols:
        report[i] = {}
        report[i]["na"] = data[i].isna().sum() / len(data)
        if "unique" in str(data[i].describe()):
            if len(data[i].unique()) <  0.25 * len(data):
                report[i]["type"] = "cat"
                report[i]["uniq"] = data[i].unique()
                report[i]["uniq_no"] = len(data[i].unique())
            else:
                report["uncertain"].append(i)
        else:
            report[i]["type"] = "num"
    return report


# Accident_Index and Datetime provide no useful information
prepro_data, logs, encs = preprop(data, to_drop=["Accident_Index", "Datetime"])  
print("\n".join(logs))


Dropped column Accident_Index as requested
Dropped column Datetime as requested
Encoded column Region
Encoded column Urban_or_Rural_Area
Encoded column X1st_Road_Class
Encoded column Road_Type
Encoded column Road_Surface_Conditions
Encoded column Weather
Encoded column High_Wind
Encoded column Lights
Encoded column Junction_Detail
Encoded column Junction_Location
Encoded column X1st_Point_of_Impact
Encoded column Driver_Journey_Purpose
Encoded column Propulsion_Code
Encoded column Vehicle_Make
Encoded column Vehicle_Category
Encoded column Vehicle_Manoeuvre
Encoded column Accident_Severity


ModuleNotFoundError: No module named 'plotly'